In [ ]:
import os
import pandas as pd
import niifiles_utils as niiut
import numpy as np

In [ ]:
TESTDATA_PATH = 'test-data/workstream-2'
X_SHAPE = 322
Y_SHAPE = 307

In [ ]:
flist = [f.split('.')[0] for f in os.listdir(TESTDATA_PATH)]
df = pd.DataFrame()
df['image_name'] = sorted(flist)
df['index'] = [157, 181, 183, 167, 198, 184, 161, 161, 193, 173, 175, 178, 200]
df.to_csv('test.csv')

In [ ]:
flist2 = sorted(os.listdir('sliced_data_test/z_axis'))
df2 = pd.DataFrame()
df2['img_file_name'] = flist2
df2['Image Name'] = df2['img_file_name'].apply(lambda x: x.split('.')[0])
df2[['Image Name', 'Slice Index']] = df2['Image Name'].str.split('_', expand=True)
df2['Slice Index'] = df2['Slice Index'].astype(int)
df2['Growth Plate Index'] = 0

In [ ]:
df2 = df2.sort_values(by=['Image Name', 'Slice Index'], ignore_index=True)

In [ ]:
df2.head(3)

In [ ]:
df2.to_csv('metadata_test/test_zaxis.csv')

In [ ]:
file = niiut.get_random_niifilename(folderpath=TESTDATA_PATH)

In [ ]:
arr = niiut.load_niifile_as_numpy('test-data/workstream-2/feaec917f3.nii')

In [ ]:
niiut.show_multiple_slices(arr, n=642)

In [ ]:
sums_norm = niiut.get_slices_sums_normalized(TESTDATA_PATH)

In [ ]:
def window_test_image(x_thr_max: float, y_thr_max: float,
                      x_shape = X_SHAPE, y_shape=Y_SHAPE,
                      folderpath = TESTDATA_PATH, outputpath='sliced_data_test/z_axis'):
    '''
    Reduce amount of background.
    '''
    
    # make a folder
    if not os.path.exists(outputpath):
        os.makedirs(outputpath)

    area_sums_norm = niiut.get_slices_sums_normalized(folderpath)
    shapes= [x_shape, y_shape]
    thrs = [x_thr_max, y_thr_max]
    for i, ax_data in enumerate(area_sums_norm):
        
        if i == 2:
            continue
            
        sums_norm_maxes = np.max(x_sums_norm, axis=0)
        # idxsort = np.argsort(x_sums_norm_sumed)
        # idxsort_shape = idxsort[-(shapes_idx[i]) + 1:-1]
        # idxmin = np.sort(idxsort_shape)
        
        # i_min = int((idxmin[0] + idxmin[-1]) // 2 - shapes_idx[i] / 2)
        # i_max = (idxmin[0] + idxmin[-1]) // 2 + shapes_idx[i] // 2
        i_min = np.where(sums_norm_maxes > thrs[i])[0][0]
        i_max = i_min + shapes[i]
        
        if i == 0:
            xmin = i_min
            xmax = i_max
        elif i == 1:
            ymin = i_min
            ymax = i_max
        
    filenames = [f.split('.')[0] for f in os.listdir(folderpath)]
    flist = [os.path.join(folderpath, file) for file in os.listdir(folderpath)]
    for i, file in enumerate(flist):
        imgarray = niiut.load_niifile_as_numpy(file)
        
        if xmin < 0:
            xmin = 0
            xmax = x_shape
        if ymin < 0:
            ymin = 0
            ymax = y_shape
        if xmax > imgarray.shape[0]:
            xmax = imgarray.shape[0]
            xmin = xmax - x_shape
        if ymax > imgarray.shape[1]:
            ymax = imgarray.shape[1]
            ymin = ymax - y_shape
                
        for z in range(imgarray.shape[2]):
            imgarray_windowed = imgarray[xmin:xmax, ymin:ymax, z]
            imgarray_windowed = imgarray_windowed.reshape((imgarray_windowed.shape[0], imgarray_windowed.shape[1], 1))
            print(imgarray_windowed.shape)
            if imgarray_windowed.shape[0] != x_shape or imgarray_windowed.shape[1] != y_shape:
                print(f'ERROR: {filenames[i]} shape: {imgarray_windowed.shape}')
                continue
            np.save(os.path.join(outputpath, filenames[i] + "_" + str(z) + '.npy'), imgarray_windowed)
            print(f"{filenames[i]}_{z}.npy saved")

In [ ]:
window_test_image(x_thr_max = 0.03, y_thr_max = 0.03)

In [ ]:
testslices = os.listdir('sliced_data_test/z_axis')

In [ ]:
len(testslices)

In [ ]:
import random
import matplotlib.pyplot as plt

randslice = random.choice(testslices)
slicearr = np.load(os.path.join('sliced_data_test/z_axis', randslice))

plt.imshow(slicearr, cmap='bone')
plt.axis('off')
plt.show()

In [ ]:
train_slices = os.listdir('sliced_data_new/z_axis')

In [ ]:
randslice = random.choice(train_slices)
slicearr = np.load(os.path.join('sliced_data_new/z_axis', randslice))

plt.imshow(slicearr, cmap='bone')
plt.axis('off')
plt.show()